In [53]:
%set_env GOOGLE_APPLICATION_CREDENTIALS=../../keys/key.json

env: GOOGLE_APPLICATION_CREDENTIALS=../../keys/key.json


In [54]:
text = """
300 g marchewki
1 ziemniak
500 g cukinii
300 g bakłażana
1/4 cebuli
2 sztuki zielonej papryczki chili*
400 ml (1 puszka) mleka kokosowego
1/2 łyżeczki mielonego kminu rzymskiego
4 cm świeżej kurkumy lub 1 łyżeczka mielonej
1 limonka
"""

# text = """250 g jogurtu naturalnego
# 2 jajka
# 1 op. cukru wanilinowego
# 170 g mąki pszennej
# 1 i 1/2 łyżeczki proszku do pieczenia
# 1/2 łyżeczki sody oczyszczonej
# masło do posmarowania patelni lub olej"""

In [55]:
import sys
sys.path.insert(0, '..')
from translate.translate import translate_text

###
# Hardcoded to prevent paid requests

translated = translate_text(text)

# translated = """300 g of carrots
# 1 potato
# 500 g of zucchini
# 300 g of eggplant
# 1/4 of the onion
# 2 pieces of green chili pepper*
# 400 ml (1 can) of coconut milk
# 1/2 teaspoon ground cumin
# 4 cm of fresh turmeric or 1 teaspoon of ground
# 1 lime"""


Translated text: 
300 g of carrots
1 potato
500 g of zucchini
300 g of eggplant
1/4 of the onion
2 pieces of green chili pepper *
400 ml (1 can) of coconut milk
1/2 teaspoon ground cumin
4 cm of fresh turmeric or 1 teaspoon of ground
1 lime



In [56]:
import nltk
import re
from translate.translate import remove_alternatives
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/rafcio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rafcio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rafcio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [57]:


porter = PorterStemmer()
lemm = nltk.WordNetLemmatizer()


def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words("english")
    content = [w for w in text if w.lower() not in stopwords]
    return content


pattern = """
NP: {(<CD>(<NN>|<JJ>)?)}(<JJ>|<NN>)+
    {(<JJ>|<NN>)+}
"""

parser = nltk.RegexpParser(pattern)

ingredients = translated.split("\n")

ingredients_list = []

for i in ingredients:
    sentence = remove_alternatives(i)
    words = nltk.word_tokenize(sentence)
    words = [w.lower() for w in words]
    words = remove_stopwords(words)
    words = [porter.stem(w) for w in words]
    words = [lemm.lemmatize(w) for w in words]
    tags = nltk.pos_tag(words)
    parsed = parser.parse(tags)

    if(len(parsed) > 1):
        amount = parsed[0]

        value = amount[0][0]
        if(len(amount) == 2):
            [unit, _] = parsed[0][1]
        else:
            unit = "piec"

        name = " ".join([v for [v, _] in parsed[1]])
        dictionary = {"name": name, "unit": unit, "value": value}
        ingredients_list.append(dictionary)
        print(dictionary)


{'name': 'carrot', 'unit': 'g', 'value': '300'}
{'name': 'potato', 'unit': 'piec', 'value': '1'}
{'name': 'zucchini', 'unit': 'g', 'value': '500'}
{'name': 'eggplant', 'unit': 'g', 'value': '300'}
{'name': 'onion', 'unit': 'piec', 'value': '1/4'}
{'name': 'green chili pepper', 'unit': 'piec', 'value': '2'}
{'name': 'coconut milk', 'unit': 'ml', 'value': '400'}
{'name': 'ground cumin', 'unit': 'teaspoon', 'value': '1/2'}
{'name': 'fresh turmer', 'unit': 'cm', 'value': '4'}
{'name': 'lime', 'unit': 'piec', 'value': '1'}


In [58]:
import requests
import json
from jinja2 import Template

api_url = "https://api.nal.usda.gov/fdc/v1/foods/search"


def search_for_product(ingredient):
    url = api_url + "?query=" + \
        ingredient["name"] + \
        "&api_key=852MDBUJFnEZ6JVmwpZl1SO4umbsVfNxYa1Sb0TK&pageSize=1"

    response = requests.get(url)

    if(response.status_code == 200):
        body = json.loads(response.text)
        foods = body["foods"]
        if(len(foods) > 0):
            food = foods[0]
    else:
        print("BŁĄÐ", response)

    return food

products = [search_for_product(i) for i in ingredients_list]

In [60]:
template_content = """
<h1>Przepis:</h1>
<p><pre>{{ recipe }}</pre></p>

<h1>Podsumowanie</h1>
<table>
    <tr>
        <th>Składnik</th>
        <th>Wartość</th>
        <th>Jednostka</th>
    </tr>
    {% for k, v in summary.items() %}
        <tr>
            <td> {{ k }} </td>
            <td> {{ v }} </td>
            <td> G </td>
        </tr>
    {% endfor %}
</table>

<h1>Składniki</h1>
{% for product in data %}
    <h2>{{ product[0] }}</h2>
    <h3>{{ product[1]["name"] }}, {{ product[1]["value"] }} {{ product[1]["unit"] }}</h3>
    <table>
        <tr>
            <th>Składnik</th>
            <th>Wartość</th>
            <th>Jednostka</th>
        </tr>
        {% for i in product[2] %}
            <tr>
                <td> {{ i["nutrientName"] }} </td>
                <td> {{ i["value"] }} </td>
                <td> {{ i["unitName"] }} </td>
            </tr>
        {% endfor %}
    </table>
{% endfor %}"""

template = Template(template_content)

In [61]:

import requests
import json
from IPython.display import HTML
from jinja2 import Template
from functools import reduce

result = []
nutrients_list = []

macros = ["Protein", "Total lipid (fat)", "Carbohydrate, by difference", "Energy"]

keys = ["nutrientName", "value", "unitName"]


for [pl_name, ingredient, product] in zip(text.split("\n"), ingredients_list, products):
    nutrients = product["foodNutrients"]
    nutrients = [{k: i[k] for k in keys}
                 for i in nutrients if i["nutrientName"] in macros]
    nutrients_list.append(nutrients)
    result.append([pl_name, ingredient, nutrients])


flattened_nutrients_list = [
    item for sublist in nutrients_list for item in sublist]

summary = {m: sum(
    [x["value"] for x in flattened_nutrients_list if x["nutrientName"] == m]) for m in macros}

HTML(template.render(data=result, summary=summary, recipe=text))


Składnik,Wartość,Jednostka
Protein,40.45,G
Total lipid (fat),45.620000000000005,G
"Carbohydrate, by difference",260.64,G
Energy,3250.0,G
Składnik,Wartość,Jednostka
Protein,0.93,G
Total lipid (fat),0.24,G
"Carbohydrate, by difference",9.58,G
Energy,41.0,KCAL
Składnik,Wartość,Jednostka
